In [8]:
import pandas as pd
import py7zr

In [9]:
# Extract tsv files
with py7zr.SevenZipFile('data/train.tsv.7z', mode='r') as z:
    z.extract(path="data")
with py7zr.SevenZipFile('data/test.tsv.7z', mode='r') as z:
    z.extract(path="data")

In [10]:
data_df = pd.read_csv("data/train.tsv", sep='\t')

In [11]:
data_df = data_df.drop('shipping',axis=1)
data_df = data_df.dropna()
# Only using the instances reuired for test/train
data_df = data_df[:200000]
data_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,item_description
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,Adorable top with a hint of lace and a key hol...
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,Size small but straps slightly shortened to fi...
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,You get three pairs of Sophie cheer shorts siz...
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,Girls Size small Plus green. Three shorts total.


In [12]:
# Combining descriptive data to one column
data_df['description'] = data_df[['name', 'brand_name', 'item_description']].agg(' '.join, axis=1)
data_df = data_df.drop(['name', 'brand_name', 'item_description', 'train_id'], axis=1)
data_df

,item_condition_id,category_name,price,description
1,3,Electronics/Computers & Tablets/Components & P...,52.0,Razer BlackWidow Chroma Keyboard Razer This ke...
2,1,Women/Tops & Blouses/Blouse,10.0,AVA-VIV Blouse Target Adorable top with a hint...
6,3,Women/Swimwear/Two-Piece,64.0,Acacia pacific tides santorini top Acacia Swim...
7,3,Sports & Outdoors/Apparel/Girls,6.0,Girls cheer and tumbling bundle of 7 Soffe You...
8,3,Sports & Outdoors/Apparel/Girls,19.0,Girls Nike Pro shorts Nike Girls Size small Pl...
...,...,...,...,...
350139,3,Electronics/Video Games & Consoles/Games,4.0,Xbox 360 gears of War 2 Xbox Have a used game ...
350140,1,"Women/Athletic Apparel/Pants, Tights, Leggings",75.0,Nwt os lularoe Skelicorns black unicorn LuLaRo...
350141,3,"Women/Tops & Blouses/Tank, Cami",8.0,Pink tropical tank PINK All clothes need to go...
350143,1,"Women/Skirts/Straight, Pencil",8.0,Free Ship NWT purple pink pencil skirt FOREVER...


In [13]:
from rake_nltk import Rake
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Rakes categories that exist in cat_attribs, outputs a String
def rake_implement(x):
    r = Rake()
    string = ""
    r.extract_keywords_from_text(x)
    # temp is list of keywords from the above function
    temp = r.get_ranked_phrases()

    # concats the list items to a string
    for i in temp:
        string = string + i + " "
    return string

# Uses bag of words to obtain all keywords, then merges both dataframes
def bag_of_words(df, cat, feat=8000):
    tfidf = TfidfVectorizer(strip_accents='ascii', max_features=feat, max_df=0.95, min_df=1)
    # Applying rake algorithm to string
    df[cat] = df[cat].apply(lambda x: rake_implement(x))
    # Fitting and transforming the raked string
    tfidf.fit(df[cat])
    X = tfidf.transform(df[cat])
    # Drop the old column (since df will be merged)
    df = df.drop([cat], axis=1)

    X_df = pd.DataFrame(X.toarray(), columns=sorted(tfidf.vocabulary_))
    return pd.merge(df, X_df, left_index=True, right_index=True)

In [14]:
data_df = bag_of_words(data_df, 'description')
data_df = bag_of_words(data_df, 'category_name')
data_df

,item_condition_id,price_x,00,000,001,007,01,02,03,04,...,women_y,woodcuts,woodwind,wool_y,work_y,wrap_y,wraps_y,writing_y,yoga_y,zip_y
1,3,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.224474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.168717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.267272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.192214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114215,2,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114217,3,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114218,2,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.189734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114219,4,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.123439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
label = data_df['price_x'].copy()
features = data_df.drop('price_x', axis=1)
features

,item_condition_id,00,000,001,007,01,02,03,04,05,...,women_y,woodcuts,woodwind,wool_y,work_y,wrap_y,wraps_y,writing_y,yoga_y,zip_y
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.224474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.168717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.267272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.192214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114215,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114217,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114218,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.189734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114219,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.123439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# Identifying each attribute for pipeline use
num_attribs = ['item_condition_id']
cat_attribs = []

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),   #returns a dense matrix
    ('cat', OrdinalEncoder(), cat_attribs) #returns a sparse matrix
])

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.15, random_state=42)

In [18]:
full_pipeline = full_pipeline.fit(X_train)
X_train_prep = full_pipeline.transform(X_train)
X_test_prep = full_pipeline.transform(X_test)

In [19]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=21)

knn.fit(X_train_prep, y_train)

KNeighborsRegressor(n_neighbors=21)

In [20]:
predictions = knn.predict(X_test_prep)

In [21]:
X_test['actual_price'] = y_test
X_test['new_price'] = predictions
X_test.tail(50)

C:\Users\Willi\AppData\Local\Temp\ipykernel_17516\4015258132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['actual_price'] = y_test
C:\Users\Willi\AppData\Local\Temp\ipykernel_17516\4015258132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['new_price'] = predictions


,item_condition_id,00,000,001,007,01,02,03,04,05,...,woodwind,wool_y,work_y,wrap_y,wraps_y,writing_y,yoga_y,zip_y,actual_price,new_price
71838,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,19.0,21.095238
21706,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,7.0,60.428571
74344,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,26.0,21.095238
20152,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,36.0,21.095238
54273,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,41.0,60.428571
101310,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,20.0,27.190476
8159,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,41.0,27.190476
64382,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,10.0,27.190476
76655,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,8.0,27.190476
23334,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,13.0,27.190476
